In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
print(tf.__version__)
import keras

1.15.2


Using TensorFlow backend.


In [3]:
from keras.datasets import mnist
(X_train,y_train),(X_test,y_test)=mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [4]:

X_train=X_train/255
X_test=X_test/255

X_train=X_train.reshape(60000,28,28,1)
X_test=X_test.reshape(10000,28,28,1)

In [5]:

from keras.utils import to_categorical
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)

In [6]:
from keras.preprocessing.image import ImageDataGenerator
datagen=ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1,rotation_range=10)
datagen.fit(X_train)

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten
from keras.optimizers import Adam


model=Sequential()
model.add(Conv2D(filters=30,kernel_size=(3,3),input_shape=(28,28,1),activation='relu',name='input_1'))
model.add(MaxPooling2D(pool_size=(2,2),name='pool_1'))

model.add(Conv2D(filters=60,kernel_size=(3,3),activation='relu',name='Conv2'))
model.add(MaxPooling2D(pool_size=(2,2),name='pool_2'))

model.add(Flatten())

model.add(Dense(512,activation='relu',name='dense_1'))
model.add(Dense(10,activation='softmax',name='output'))

adam=Adam(lr=0.001)

model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [8]:
history=model.fit_generator(datagen.flow(X_train,y_train,batch_size=200),steps_per_epoch=500,epochs=2,validation_data=(X_test,y_test),shuffle=1)


Epoch 1/2
500/500 [==============================] - 108s 215ms/step - loss: 0.3043 - accuracy: 0.9051 - val_loss: 0.0479 - val_accuracy: 0.9839
Epoch 2/2
500/500 [==============================] - 108s 215ms/step - loss: 0.0969 - accuracy: 0.9701 - val_loss: 0.0383 - val_accuracy: 0.9870


In [9]:
model.save('digit.h5')

In [10]:
from keras.models import load_model
model = load_model("digit.h5")
session = keras.backend.get_session()

saver = tf.train.Saver()
saver.save(session, "digit.ckpt")

'digit.ckpt'

In [11]:
saver = tf.train.import_meta_graph('digit.ckpt.meta', clear_devices=True)
graph = tf.get_default_graph()
input_graph_def = graph.as_graph_def()
sess = tf.Session()
saver.restore(sess, "./digit.ckpt")

INFO:tensorflow:Restoring parameters from ./digit.ckpt


In [12]:
from keras import backend as K
import tensorflow as tf
K.set_learning_phase(0)
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(sess,
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 78 variables.
INFO:tensorflow:Converted 78 variables to const ops.


In [13]:
output_graph="digits.pb"
with tf.gfile.GFile(output_graph, "wb") as f:
    f.write(frozen_graph.SerializeToString())
 


In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (Conv2D)             (None, 26, 26, 30)        300       
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 13, 13, 30)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 11, 11, 60)        16260     
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 5, 5, 60)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               768512    
_________________________________________________________________
output (Dense)               (None, 10)               

In [15]:
print('Inputs:',model.inputs)
print('Outputs:',model.outputs)

Inputs: [<tf.Tensor 'input_1_input_1:0' shape=(?, 28, 28, 1) dtype=float32>]
Outputs: [<tf.Tensor 'output_1/Softmax:0' shape=(?, 10) dtype=float32>]


In [16]:
from tensorflow.tools.graph_transforms import TransformGraph
TRANSFORMS=['remove_nodes(op=Indentity)','fold_constants(ignore_errors=True)','merge_duplicate_nodes',
            'strip_unused_nodes','fold_batch_norms']
optimized_graph_def=TransformGraph(frozen_graph,['input_1_input_1:0'],['output_1/Softmax:0'],TRANSFORMS)

In [17]:
output_graph="digits_optimized.pb"
with tf.gfile.GFile(output_graph, "wb") as f:
    f.write(optimized_graph_def.SerializeToString())
 


In [18]:
sess.close()

In [19]:
frozen_graph="digits_optimized.pb"
with tf.gfile.GFile(frozen_graph, "rb") as f:
      graph_def = tf.GraphDef()
      graph_def.ParseFromString(f.read())
 
with tf.Graph().as_default() as graph:
      tf.import_graph_def(graph_def,
                          input_map=None,
                          return_elements=None,
                          name="")

In [20]:
y_pred = graph.get_tensor_by_name("output_1/Softmax:0")
## Let's feed the images to the input placeholders
x= graph.get_tensor_by_name("input_1_input_1:0")
sess= tf.Session(graph=graph)
### Creating the feed_dict that is required to be fed to calculate y_pred 
feed_dict_testing = {x: X_test[:20]}
result=sess.run(y_pred, feed_dict=feed_dict_testing)
print(result)

[[1.33368703e-08 2.60209049e-07 6.89802109e-05 1.90243245e-05
  9.34931368e-11 5.88880084e-07 9.66818747e-13 9.99899745e-01
  1.20574225e-06 1.01708274e-05]
 [1.05050976e-05 5.25212045e-06 9.99980688e-01 2.21065164e-07
  1.93806358e-08 1.06841411e-11 1.48166168e-06 2.41437426e-09
  1.84831003e-06 9.83309967e-09]
 [3.65540254e-05 9.99435008e-01 4.47676075e-06 6.78062548e-08
  3.41784063e-04 1.06231173e-05 1.38230796e-04 2.24160158e-05
  7.17951389e-06 3.67235089e-06]
 [6.32182777e-01 8.79646507e-08 3.17238184e-04 1.72659711e-04
  8.57481427e-05 1.03434068e-04 3.54803234e-01 1.57919487e-06
  8.51496030e-03 3.81818390e-03]
 [9.98329241e-09 2.53682817e-08 2.59795332e-08 1.11492771e-09
  9.99834895e-01 3.08623327e-09 1.62089265e-07 1.39282463e-06
  7.22342932e-08 1.63356439e-04]
 [8.83884059e-06 9.99932766e-01 2.25028288e-07 1.65674197e-09
  2.04196749e-05 9.50581182e-07 2.80950790e-05 4.14373199e-06
  3.93899018e-06 5.37471863e-07]
 [8.34610381e-10 5.07205755e-07 7.04906370e-07 1.51514943e